In [2]:
from transformers import BertTokenizer, TFBertForSequenceClassification
import string
from nltk.corpus import stopwords
import pandas as pd
import re

# Load pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")



In [3]:
data = pd.read_csv("dataset.csv")

In [4]:
from keras.preprocessing.text import text_to_word_sequence
stop_words = set(stopwords.words('english'))

# remove "(<.*?>)" markup
data['text'] = data['text'].apply(lambda x: re.sub('(<.*?>)', ' ', x))

# remove punctuation marks 
data['text'] = data['text'].apply(lambda x: re.sub('[,\.!?:()"]', '', x))
    
# remove whitespace
data['text'] = data['text'].apply(lambda x: x.strip())
    
# convert to lower
data['text'] = data['text'].apply(lambda x: x.lower())

# stopwords removal
words = data['text'].apply(lambda x: text_to_word_sequence(x))
filtered_words = words.apply(lambda x: [w for w in x if w not in stop_words])
data['text'] = filtered_words.apply(lambda x: " ".join(x))

In [5]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(data['text'],data['classes'],random_state=1,test_size=0.2,shuffle=True)

In [6]:
max_len= 256
# Tokenize and encode the sentences
X_train_encoded = tokenizer.batch_encode_plus(X_train.tolist(),
                                              padding=True, 
                                              truncation=True,
                                              max_length = max_len,
                                              return_tensors='tf')

X_test_encoded = tokenizer.batch_encode_plus(X_test.tolist(),
                                              padding=True, 
                                              truncation=True,
                                              max_length = max_len,
                                              return_tensors='tf')

In [7]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
from keras.optimizers import Adam
from keras.losses import SparseCategoricalCrossentropy
from keras.metrics import SparseCategoricalAccuracy

# Compile the model with an appropriate optimizer, loss function, and metrics
optimizer = Adam(learning_rate=2e-5)
loss = SparseCategoricalCrossentropy(from_logits=True)
metric = SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [9]:

# Train the model
history = model.fit(
    [X_train_encoded['input_ids'], X_train_encoded['token_type_ids'], X_train_encoded['attention_mask']],
    y_train,
    validation_data=(
      [X_test_encoded['input_ids'], X_test_encoded['token_type_ids'], X_test_encoded['attention_mask']],y_test),
    batch_size=32,
    epochs=3
)

Epoch 1/3

1600/1600 [==============================] - 45960s 29s/step - loss: 0.0420 - accuracy: 0.9891 - val_loss: 0.0241 - val_accuracy: 0.9949
Epoch 2/3
1600/1600 [==============================] - 35769s 22s/step - loss: 0.0224 - accuracy: 0.9945 - val_loss: 0.0154 - val_accuracy: 0.9954
Epoch 3/3
1600/1600 [==============================] - 34511s 22s/step - loss: 0.0151 - accuracy: 0.9955 - val_loss: 0.0194 - val_accuracy: 0.9954


In [23]:
model.save_pretrained('.\\bert')

In [24]:
model_saved = TFBertForSequenceClassification.from_pretrained('.\\bert')

Some layers from the model checkpoint at .\bert were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at .\bert.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [25]:
y_predict = model_saved.predict([X_test_encoded['input_ids'], X_test_encoded['token_type_ids'], X_test_encoded['attention_mask']])

  3/400 [..............................] - ETA: 31:16

KeyboardInterrupt: 